In [1]:
# Importing all the necessary libraries

import pandas as pd 
from parsel import Selector
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException  # This line is important
import time
import random
import os
import re

opts = Options ()

In [ ]:

# Initiating the chrome driver


driver = webdriver.Chrome()

# Function to ensure all key data fields have a value

def validate_field(field):
    #if field is present pass if field:
    if field:
        pass
    #if field is not present print text else:
    else:
        field = 'No results'
    return field

#driver get method() will navigate to a page given by the URL address.
driver.get("https://www.linkedin.com/")

#locate email form by_class_name
username = driver.find_element(By.ID, 'session_key')

#send_keys() to simulate key strokes
username.send_keys('2023mlpr@gmail.com')

#sleep for 0.5 seconds
sleep(0.5)

#Locate password form by_class_name
password = driver.find_element(By.ID, 'session_password')

#send_keys() to simulate key strokes
password.send_keys('MLPR2023')
sleep(0.5)

#Locate submit button by_xpath
sign_in_button = driver.find_element(By.XPATH,'//*[@type="submit"]')

#.click() to mimic button click
sign_in_button.click()

sleep(10)

In [ ]:

# Extracting profiles based on the keyword taken --- Here the keyword taken is Python + Developer

sleep(5)
Jobdata = []
lnks = []
for x in range(0,20,10):
    driver.get(f'https://www.google.com/search?q=site:linkedin.com/in/+AND+%22Python+Developer%22+AND+%22Delhi%22&rlz=1C1CHZO_enIN1023IN1023&ei=jPaIY-mEGM6cseMPyZaNmAo&start={x}')
    time.sleep(random.uniform(2.5,4.9))
    try:
        linkedin_url = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[@jsname='UWckNb']"))).get_attribute("href")
        linkedin_urls = [my_elem.get_attribute("href") for my_elem in WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//a[@jsname='UWckNb']")))]
        lnks.append(linkedin_urls)
    except TimeoutException:
        print("TimeoutException: Elements not found within the specified time.")


In [ ]:
# Block for all the functions

# Checks whether the string is empty or not

def emptyChecker(string):
    if string:
        return string
    else:
        return "nan"

# Used for waiting an element to load

def wait_for_element_to_load(by=By.CLASS_NAME, name="pv-top-card", base=None):
    base = base or driver
    return WebDriverWait(base,5).until(
            EC.presence_of_element_located(
                (
                    by,
                    name
                )
            )
        )

# Function to extract description

def extract_description(data):
    # Split the data by lines
    lines = data.strip().split('\n')
    
    # Extract the description part (after date)
    description = '\n'.join(lines[4:])
    description = removingExtraLine(description)
    return description

def removingExtraLine(descrip):
    
    # Define a regular expression pattern to match a date range format
    date_pattern = r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4} - (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4} · \d+ mos\b"

    # Use re.search() to find the date range pattern in the first line
    match = re.search(date_pattern, descrip)

    # If a match is found in the first line, remove that line
    if match and match.start() == 0:
        lines = descrip.split('\n', 1)  # Split the data into lines, starting from the first newline
        if len(lines) > 1:
            descrip = lines[1]  # Keep the data after the first line
            
    return descrip

# Extracts the entire education section of the user

def educationInfo(url):
    try:
        url = os.path.join(url,"details/education")
        driver.get(url)
        main = wait_for_element_to_load(by = By.TAG_NAME, name = "main")
        main_list = wait_for_element_to_load(name="pvs-list", base=main)
        education_from_date = ""
        education_to_date = ""
        education_description = ""
        education_degree = ""
        education_institution_name = ""
        
        for position in main_list.find_elements(By.CLASS_NAME,"pvs-entity"):
                institution_logo_elem, position_details = position.find_elements(By.XPATH,"*")
                # company elem
                institution_linkedin_url = institution_logo_elem.find_element(By.XPATH,"*").get_attribute("href")

                # position details
                position_details_list = position_details.find_elements(By.XPATH,"*")
                position_summary_details = position_details_list[0] if len(position_details_list) > 0 else None
                position_summary_text = position_details_list[1] if len(position_details_list) > 1 else None
                outer_positions = position_summary_details.find_element(By.XPATH,"*").find_elements(By.XPATH,"*")

                institution_name = outer_positions[0].find_element(By.TAG_NAME,"span").text
                degree = outer_positions[1].find_element(By.TAG_NAME,"span").text

                if len(outer_positions) > 2:
                    times = outer_positions[2].find_element(By.TAG_NAME,"span").text
                    dates = times.split('-')
                    from_date = dates[0].strip()
                    to_date = dates[1].strip()
                else:
                    from_date = None
                    to_date = None

                description = position_summary_text.text if position_summary_text else ""
                print()
                education_from_date = education_from_date + "-:-" + emptyChecker(from_date)
                education_to_date = education_to_date + "-:-" + emptyChecker(to_date)
                education_description = education_description + "-:-" + emptyChecker(description)
                education_degree = education_degree + "-:-" + emptyChecker(degree)
                education_institution_name = education_institution_name + "-:-" + emptyChecker(institution_name)
        return [education_from_date,education_to_date,education_description,education_degree, education_institution_name]
            
    except:
        
        return ["nan", "nan", "nan", "nan", "nan"]
    
# Extracts the entire experience section of the user

def experienceSection(url):

    try:    
        url = os.path.join(url, "details/experience")
        driver.get(url)
        main = wait_for_element_to_load(by=By.TAG_NAME, name="main")
        main_list = wait_for_element_to_load(name="pvs-list", base=main)

        experience_section_position_title = ""
        experience_section_from_date = ""
        experience_section_to_date = ""
        experience_section_duration = ""
        experience_section_description = ""
        experience_section_company = ""
        experience_section_location = ""

        for position in main_list.find_elements(By.XPATH,"li"):
            position = position.find_element(By.CLASS_NAME,"pvs-entity")
            company_logo_elem, position_details = position.find_elements(By.XPATH,"*")

            # company elem
            company_linkedin_url = company_logo_elem.find_element(By.XPATH,"*").get_attribute("href")

            # position details
            position_details_list = position_details.find_elements(By.XPATH,"*")
            position_summary_details = position_details_list[0] if len(position_details_list) > 0 else None
            position_summary_text = position_details_list[1] if len(position_details_list) > 1 else None
            outer_positions = position_summary_details.find_element(By.XPATH,"*").find_elements(By.XPATH,"*")

            if len(outer_positions) == 4:
                position_title = outer_positions[0].find_element(By.TAG_NAME,"span").text
                company = outer_positions[1].find_element(By.TAG_NAME,"span").text
                work_times = outer_positions[2].find_element(By.TAG_NAME,"span").text
                location = outer_positions[3].find_element(By.TAG_NAME,"span").text
            elif len(outer_positions) == 3:
                if "·" in outer_positions[2].text:
                    position_title = outer_positions[0].find_element(By.TAG_NAME,"span").text
                    company = outer_positions[1].find_element(By.TAG_NAME,"span").text
                    work_times = outer_positions[2].find_element(By.TAG_NAME,"span").text
                    location = ""
                else:
                    position_title = ""
                    company = outer_positions[0].find_element(By.TAG_NAME,"span").text
                    work_times = outer_positions[1].find_element(By.TAG_NAME,"span").text
                    location = outer_positions[2].find_element(By.TAG_NAME,"span").text

            times = work_times.split("·")[0].strip() if work_times else ""
            duration = work_times.split("·")[1].strip() if len(work_times.split("·")) > 1 else None

            from_date = " ".join(times.split(" ")[:2]) if times else ""
            to_date = " ".join(times.split(" ")[3:]) if times else ""

            if position_summary_text and len(position_summary_text.find_element(By.CLASS_NAME,"pvs-list").find_element(By.CLASS_NAME,"pvs-list").find_elements(By.XPATH,"li")) > 1:
                descriptions = position_summary_text.find_element(By.CLASS_NAME,"pvs-list").find_element(By.CLASS_NAME,"pvs-list").find_elements(By.XPATH,"li")
                for description in descriptions:
                    res = description.find_element(By.TAG_NAME,"a").find_elements(By.XPATH,"*")
                    position_title_elem = res[0] if len(res) > 0 else None
                    work_times_elem = res[1] if len(res) > 1 else None
                    location_elem = res[2] if len(res) > 2 else None
                    location = location_elem.find_element(By.XPATH,"*").text if location_elem else None
                    position_title = position_title_elem.find_element(By.XPATH,"*").find_element(By.TAG_NAME,"*").text if position_title_elem else ""
                    work_times = work_times_elem.find_element(By.XPATH,"*").text if work_times_elem else ""
                    times = work_times.split("·")[0].strip() if work_times else ""
                    duration = work_times.split("·")[1].strip() if len(work_times.split("·")) > 1 else None
                    from_date = " ".join(times.split(" ")[:2]) if times else ""
                    to_date = " ".join(times.split(" ")[3:]) if times else ""

                    if description:
                        text = description.text
                        description = extract_description(text)
                    experience_section_position_title = experience_section_position_title + "-:-" + emptyChecker(position_title)
                    experience_section_from_date = experience_section_from_date + "-:-" + emptyChecker(from_date)
                    experience_section_to_date = experience_section_to_date + "-:-" + emptyChecker(to_date)
                    experience_section_duration = experience_section_duration + "-:-" + emptyChecker(duration)
                    experience_section_description = experience_section_description + "-:-" + emptyChecker(description)
                    experience_section_company = experience_section_company + "-:-" + emptyChecker(company)
                    experience_section_location = experience_section_location + "-:-" + emptyChecker(location)
            else:
                description = position_summary_text.text if position_summary_text else ""
                experience_section_position_title = experience_section_position_title + "-:-" + emptyChecker(position_title)
                experience_section_from_date = experience_section_from_date + "-:-" + emptyChecker(from_date)
                experience_section_to_date = experience_section_to_date + "-:-" + emptyChecker(to_date)
                experience_section_duration = experience_section_duration + "-:-" + emptyChecker(duration)
                experience_section_description = experience_section_description + "-:-" + emptyChecker(description)
                experience_section_company = experience_section_company + "-:-" + emptyChecker(company)
                experience_section_location = experience_section_location + "-:-" + emptyChecker(location)
        return [experience_section_position_title, experience_section_from_date, experience_section_to_date,experience_section_description , experience_section_duration, experience_section_company, experience_section_location]
    except:
        return ["nan","nan", "nan", "nan", "nan",  "nan", "nan"]


In [ ]:
profile_data = []
for x in lnks:
    newProfile = {}
    for i in x:
        # Get the profile URL
        driver.get(i)
        time.sleep(random.uniform(2.5, 4.9))

        # Assigning the source code for the web page to a variable
        page_source = driver.page_source
        sel = Selector(text=page_source)
        
        # Extract name
        name_element = driver.find_element(By.XPATH, '//h1[contains(@class, "text-heading-xlarge inline t-24 v-align-middle break-words")]')
        name = name_element.text.strip() if name_element else "nan"

        # Extract job title
        job_title_element = driver.find_element(By.XPATH, '//div[contains(@class, "text-body-medium break-words")]')
        job_title = job_title_element.text.strip() if job_title_element else "nan"

        #Extract about section
        about = driver.find_element(By.XPATH,'//div[contains(@class, "inline-show-more-text         inline-show-more-text--is-collapsed         inline-show-more-text--is-collapsed-with-line-clamp                            full-width")]').text
        about = about.text.strip() if about else "nan"

        #Extracting the education
        education = educationInfo(i)

        #Extracting the experience
        experience = experienceSection(i)

        newProfile["name"] = name_element
        newProfile["currentJobTitle"] = job_title
        newProfile["about"] = about
        newProfile["education_from_date"] = education[0]
        newProfile["education_to_date"] = education[1]
        newProfile["education_description"] = education[2]
        newProfile["education_degree"] = education[3]
        newProfile["education_institution_name"] = education[4]

        newProfile["experience_section_position_title"] = experience[0]
        newProfile["experience_section_from_date"] = experience[1]
        newProfile["experience_section_to_date"] = experience[2]
        newProfile["experience_section_description"] = experience[3]
        newProfile["experience_section_duration"] = experience[4]
        newProfile["experience_section_company"] = experience[5]
        newProfile["experience_section_location"] = experience[6]

        